In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [2]:
class network_structure():
    def __init__(self, arcs_nodes_matrix):
        # Structure of arcs_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an arc.
        
        assert arcs_nodes_matrix.shape[1] == 2, "We must have arcs_nodes_matrix.shape[1] == 2"
        
        self.arcs_nodes_matrix = arcs_nodes_matrix
        self.nodes = np.unique(self.arcs_nodes_matrix.flatten())
        self.arcs = np.array(range(self.arcs_nodes_matrix.shape[0])) + 1
        
        self.construct_nodes_dict()
        self.construct_arcs_dict()
        self.construct_depth_dicts()
        self.construct_height_dicts()

        return
        
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}        
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_arcs'] = list((self.arcs_nodes_matrix[:, 1] == node).nonzero()[0] + 1)
            self.nodes_dict[node]['outgoing_arcs'] = list((self.arcs_nodes_matrix[:, 0] == node).nonzero()[0] + 1)
        return

    def construct_arcs_dict(self):
        self.arcs_dict = {}

        for arc in self.arcs:
            self.arcs_dict[arc] = {}

        for node in self.nodes:
            if len(self.nodes_dict[node]['incoming_arcs']) != 0:
                for arc in self.nodes_dict[node]['incoming_arcs']:
                    self.arcs_dict[arc]['end_node'] = node

            if len(self.nodes_dict[node]['outgoing_arcs']) != 0:
                for arc in self.nodes_dict[node]['outgoing_arcs']:
                    self.arcs_dict[arc]['start_node'] = node

        for arc in self.arcs:
            self.arcs_dict[arc] = {}
            start_node = int(np.array([node for node in self.nodes if arc in self.nodes_dict[node]['outgoing_arcs']]))
            end_node = int(np.array([node for node in self.nodes if arc in self.nodes_dict[node]['incoming_arcs']]))
            self.arcs_dict[arc]['start_node'] = start_node
            self.arcs_dict[arc]['end_node'] = end_node
            self.arcs_dict[arc]['incoming_arcs'] = self.nodes_dict[start_node]['incoming_arcs']
            self.arcs_dict[arc]['outgoing_arcs'] = self.nodes_dict[end_node]['outgoing_arcs']
        return
    
#     def construct_routes_dict(self):        
#         self.routes = [[arc] for arc in self.arcs if self.arcs_dict[arc]['start_node'] ]
        
#         route_construction_finished = False
#         while route_construction_finished is False:
#             route_construction_finished = True
        
#         return
    
    
    
#         self.routes = []
#         for height in self.height_nodes[:-1]:
#             if height == 1:
#                 self.routes = [[arc] for arc in self.nodes_dict[self.destination_node]['incoming_arcs']]
# #                 print("self.routes:", self.routes)
#             else:
#                 routes_temp = []
#                 for node in self.height_nodes_dict[height]:
#                     for route_segment in self.routes:
# #                         print("route_segment:", route_segment)
#                         if route_segment[0] in self.nodes_dict[node]['outgoing_arcs']:
#                             for incoming_arc in self.nodes_dict[node]['incoming_arcs']:
# #                                 print("incoming_arc:", incoming_arc)
#                                 routes_temp.append([incoming_arc] + route_segment)
#                         else:
#                             routes_temp.append(route_segment)
#                     self.routes = routes_temp
# #                 print("self.routes:", self.routes)

#         return
        
        

In [9]:
arcs_nodes_matrix = np.array([[1, 3], [1, 2], [2, 3], [3, 4], [1, 2], [2, 4]])
netw_example = network_structure(arcs_nodes_matrix)

print("netw_example.arcs:", netw_example.arcs)
print()
print("netw_example.nodes:", netw_example.nodes)
print()
print("netw_example.arcs_dict:", netw_example.arcs_dict)
print()
print("netw_example.nodes_dict:", netw_example.nodes_dict)
print()


netw_example.arcs: [1 2 3 4 5 6]

netw_example.nodes: [1 2 3 4]

netw_example.arcs_dict: {1: {'start_node': 1, 'end_node': 3, 'incoming_arcs': [], 'outgoing_arcs': [4]}, 2: {'start_node': 1, 'end_node': 2, 'incoming_arcs': [], 'outgoing_arcs': [3, 6]}, 3: {'start_node': 2, 'end_node': 3, 'incoming_arcs': [2, 5], 'outgoing_arcs': [4]}, 4: {'start_node': 3, 'end_node': 4, 'incoming_arcs': [1, 3], 'outgoing_arcs': []}, 5: {'start_node': 1, 'end_node': 2, 'incoming_arcs': [], 'outgoing_arcs': [3, 6]}, 6: {'start_node': 2, 'end_node': 4, 'incoming_arcs': [2, 5], 'outgoing_arcs': []}}

netw_example.nodes_dict: {1: {'incoming_arcs': [], 'outgoing_arcs': [1, 2, 5]}, 2: {'incoming_arcs': [2, 5], 'outgoing_arcs': [3, 6]}, 3: {'incoming_arcs': [1, 3], 'outgoing_arcs': [4]}, 4: {'incoming_arcs': [4, 6], 'outgoing_arcs': []}}



# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [ ]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


In [ ]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, network, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_arcs = len(network.arcs)
    num_nodes = len(network.nodes)
    
    assert tau.shape[0] == num_arcs, "toll vector length must equal the number of arcs."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_arcs)
    for e in range(num_arcs):
        x_ex[num_arcs] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[num_arcs] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_arcs)
    for e in range(num_arcs):
        x_ex[num_arcs] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_ex[num_arcs] += sum(y_in[(g, e, 1)] for g in range(num_in))
    
    ## TODO: Edit below:
    
    ell_ex = np.zeros(num_arcs)
    ell_gp = np.zeros(num_arcs)
    for e in range(num_arcs):
        ell_ex[e] = a_ex[4] * (x_ex[e] ** 4) + a_ex[3] * (x_ex[e] ** 3) + a_ex[2] * (x_ex[e] ** 2) \
                    + a_ex[1] * x_ex[e] + a_ex[0]
        ell_gp[e] = a_gp[4] * (x_gp[e] ** 4) + a_gp[3] * (x_gp[e] ** 3) + a_gp[2] * (x_gp[e] ** 2) \
                    + a_gp[1] * x_gp[e] + a_gp[0]
    
    obj_E = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el) for e in range(num_arcs)) \
                + sum(y_el[(g, e, k)] * v_E_array[g] for g in range(num_el) for k in [0, 1] for e in range(num_arcs)) * ell_ex \
                + sum(y_el[(g, e, 2)] * v_E_array[g] for g in range(num_el) for e in range(num_arcs)) * ell_gp
    obj_I = sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in) for e in range(num_arcs)) \
                + sum(y_in[(g, e, 0)] * v_I_array[g] for g in range(num_in) for e in range(num_arcs)) * ell_ex \
                + sum(y_in[(g, e, 1)] * v_I_array[g] for g in range(num_in) for e in range(num_arcs)) * ell_gp
    obj_R = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el) for e in range(num_arcs)) \
                + tau * sum(y_in[(g, e, 0)] for g in range(num_in) for e in range(num_arcs))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
    print()
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print("welfare:", welfare)
    print()
    
    return welfare

    

In [ ]:
def proj_tau_B_11(tau, B, network, tau_max = 1.0, B_max = 1.0):
    
    num_arcs = len(network.arcs)
    num_nodes = len(network.nodes)
    
    assert tau.shape[0] == num_arcs, "toll vector length must equal the number of arcs."
    
    tau_feas = cp.Variable(num_arcs)
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones(num_arcs)]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(tau_feas.value, decimals=4), np.round(B_feas.value, decimals=4)


# Chinmay's Algorithm:

In [ ]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [ ]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # Variables:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for k in range(3):
            y_el[(g, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for k in range(2):
            y_in[(g, k)] = cp.Variable(1)
            
    x_ex = cp.Variable(1)
    x_gp = cp.Variable(1)
    
    # Objective:
    func = 0.0
    
    func += 1/5 * a_ex[4] * cp.power(x_ex, 5)
    func += 1/4 * a_ex[3] * cp.power(x_ex, 4)
    func += 1/3 * a_ex[2] * cp.power(x_ex, 3)
    func += 1/2 * a_ex[1] * cp.power(x_ex, 2)
    func += a_ex[0] * x_ex
    func += 1/5 * a_gp[4] * cp.power(x_gp, 5)
    func += 1/4 * a_gp[3] * cp.power(x_gp, 4)
    func += 1/3 * a_gp[2] * cp.power(x_gp, 3)
    func += 1/2 * a_gp[1] * cp.power(x_gp, 2)
    func += a_gp[0] * x_gp
    
    func += sum(y_el[(g, 1)] / v_E_array[g] for g in range(num_el)) * tau
    func += sum(y_in[(g, 0)] / v_I_array[g] for g in range(num_in)) * tau

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, k)] >= 0.0 for g in range(num_el) for k in [0, 1, 2]]
    constraints += [y_in[(g, k)] >= 0.0 for g in range(num_in) for k in [0, 1]]
    
    constraints += [sum(y_el[(g, k)] for k in range(2)) == 1.0 for g in range(num_el)]
    constraints += [sum(y_in[(g, k)] for k in range(1)) == 1.0 for g in range(num_in)]
    
    constraints += [x_ex == sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1]) \
                               + sum(y_in[(g, 0)] for g in range(num_in))]
    constraints += [x_gp == sum(y_el[(g, 2)] for g in range(num_el)) \
                               + sum(y_in[(g, 1)] for g in range(num_in))]
    
    constraints += [y_el[(g, 0)] * tau <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for k in range(3):
            y_el_values[(g, k)] = y_el[(g, k)].value[0]
            
    y_in_values = {}
    for g in range(num_in):
        for k in range(2):
            y_in_values[(g, k)] = y_in[(g, k)].value[0]

    return y_el_values, y_in_values


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 2.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
tau_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros(num_iters)
tau_perturbed = np.zeros(num_iters)
B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[i] = tau[i] + delta[i] * w_i[0]
    B_perturbed[i] = B[i] + delta[i] * w_i[1]
    
#     if tau_perturbed[i] < B_perturbed[i] or tau_perturbed[i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[i], B_perturbed[i] = proj_tau_B_11(tau_perturbed[i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[i]:", tau[i])
    print("B[i]:", B[i])
    print("tau_perturbed[i]:", tau_perturbed[i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(tau = tau[i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

    print()
    print("y_el_values:", y_el_values)
    print("y_in_values:", y_in_values)
    print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(tau = tau_perturbed[i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau_perturbed[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    
    welfare_list.append(welfare)
    
        ## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional n ,\
    ## (e.g., 3d or 4d) arrays:\n,
    
#     def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, y_el, y_in, \
#                 a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
#                 a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0]))
    
    print("welfare:", welfare)
    print("welfare_perturbed:", welfare_perturbed)
#     print("tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare):\n", tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare))
    
    tau[i+1] = tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare)
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[1] * (welfare_perturbed - welfare)
    tau[i+1], B[i+1] = proj_tau_B_11(tau[i+1], B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = tau[i-diffs_num_cols : i-1] - tau[i-diffs_num_cols+1 : i]
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[0:10]:", tau[0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)



In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_arcs_dict = {}
    counter = 1

    arcs_remaining = list(self.arcs)
    arcs_accounted = []
#         print("[arc for arc in arcs_remaining]:", [arc for arc in arcs_remaining])
    arcs_temp = [arc for arc in arcs_remaining if len(self.arcs_dict[arc]['incoming_arcs']) == 0]
    self.depth_arcs_dict[1] = arcs_temp
    arcs_remaining = [arc for arc in arcs_remaining if arc not in arcs_temp]
    arcs_accounted.extend(arcs_temp)
    counter += 1

    while arcs_remaining != []:
#             print("[arc for arc in arcs_remaining]:", [arc for arc in arcs_remaining])
        arcs_temp = [arc for arc in arcs_remaining if set(self.arcs_dict[arc]['incoming_arcs']).issubset(set(arcs_accounted))]
        self.depth_arcs_dict[counter] = arcs_temp
        arcs_remaining = [arc for arc in arcs_remaining if arc not in arcs_temp]
        arcs_accounted.extend(arcs_temp)
        counter += 1

    self.depths = list(self.depth_arcs_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.arcs_dict[arc]['start_node'] for arc in self.depth_arcs_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_arcs_dict = {}
    counter = 1

    arcs_remaining = list(self.arcs)
    arcs_accounted = []
#         print("[arc for arc in arcs_remaining]:", [arc for arc in arcs_remaining])
    arcs_temp = [arc for arc in arcs_remaining if len(self.arcs_dict[arc]['outgoing_arcs']) == 0]
    self.height_arcs_dict[1] = arcs_temp
    arcs_remaining = [arc for arc in arcs_remaining if arc not in arcs_temp]
    arcs_accounted.extend(arcs_temp)
    counter += 1

    while arcs_remaining != []:
#             print("[arc for arc in arcs_remaining]:", [arc for arc in arcs_remaining])
        arcs_temp = [arc for arc in arcs_remaining if set(self.arcs_dict[arc]['outgoing_arcs']).issubset(set(arcs_accounted))]
        self.height_arcs_dict[counter] = arcs_temp
        arcs_remaining = [arc for arc in arcs_remaining if arc not in arcs_temp]
        arcs_accounted.extend(arcs_temp)
        counter += 1

    self.heights = list(self.height_arcs_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.arcs_dict[arc]['end_node'] for arc in self.height_arcs_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
# for i in range(2):
#     constraints += [x[i] >= 2]
# constraints += [x[i] >=2 for i in range(2)]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
